# Demonstrations

In [ ]:
using IsingModel
using BenchmarkTools
using Distributions
import Graphs
using GraphPlot: gplot
using LinearAlgebra
using Plots: plot, plot!
using Random
using Revise

In [ ]:
### When using NetworkX, uncomment the following lines.
# using Conda
# using PyCall
# using SparseArrays
# Conda.add("scipy")     # Make `Conda.add` run once the first time.
# Conda.add("networkx")
# const nx = pyimport("networkx")

## Ref: https://github.com/JuliaPy/PyCall.jl/issues/204
# const scipy_sparse_find = pyimport("scipy.sparse")["find"]
# function mysparse(Apy::PyObject)
#     IA, JA, SA = scipy_sparse_find(Apy)
#     return sparse(Int[i + 1 for i in IA], Int[i + 1 for i in JA], SA)
# end

In [ ]:
const N = 64  # The number of nodes
const SIDE_LENGTH = (Int ∘ ceil ∘ sqrt)(N)

### A way using the Graphs.jl library
G = Graphs.grid((SIDE_LENGTH, SIDE_LENGTH), periodic=true)
adjacencyMatrix = map(Graphs.adjacency_matrix(G)) do c
    ifelse(c == 0, 0, -1)
end

### Generate a square lattice with the periodic boundary condition by NetworkX.
# G = nx.grid_2d_graph(SIDE_LENGTH, SIDE_LENGTH, periodic=true)
# nx.set_edge_attributes(G, values=-1, name="weight")
# adjacencyMatrix = mysparse(nx.adjacency_matrix(G))

bias = zeros(N)
const INITIAL_CONFIGURATION = 2 .* rand(Bernoulli(0.5), N) .- 1
spinSystem = SpinSystem(INITIAL_CONFIGURATION, adjacencyMatrix, bias)

In [ ]:
gplot(G)

In [ ]:
const MAX_STEPS = N^2
const INITIAL_TEMPERATURE = float(N)
const FINAL_TEMPERATURE = 0.0

#annealingSchedule(n) = (FINAL_TEMPERATURE - INITIAL_TEMPERATURE) / MAX_STEPS * n + INITIAL_TEMPERATURE
annealingSchedule(n) = INITIAL_TEMPERATURE^(-n)

function runAnnealer(Algorithm::Type{<:IsingModel.UpdatingAlgorithm}, spinSystem::SpinSystem, INITIAL_TEMPERATURE=-1.0)::Vector{Float64}
    if INITIAL_TEMPERATURE < 0.0
        return map(calcEnergy, takeSamples!(Algorithm(deepcopy(spinSystem)), MAX_STEPS, annealingSchedule))
    else
        return map(calcEnergy, takeSamples!(Algorithm(deepcopy(spinSystem), INITIAL_TEMPERATURE), MAX_STEPS, annealingSchedule))
    end
end

In [ ]:
@benchmark runAnnealer(GlauberDynamics, spinSystem, INITIAL_TEMPERATURE)

In [ ]:
plot(xlabel="MC steps", ylabel="Energy")
plot!(runAnnealer(AsynchronousHopfieldNetwork, spinSystem), label="Hopfield")
plot!(runAnnealer(GlauberDynamics, spinSystem, INITIAL_TEMPERATURE), label="Glauber")
plot!(runAnnealer(MetropolisMethod, spinSystem, INITIAL_TEMPERATURE), label="Metropolis")